In [ ]:
# --- 1. INSTALL ALL DEPENDENCIES ---
print("Installing dependencies...")
!pip install datasets sentence-transformers umap-learn hdbscan pandas numpy

import pandas as pd
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap
import hdbscan
import time
from huggingface_hub import notebook_login, get_token

# --- 2. AUTHENTICATE SESSION ---
# This will prompt you for your API token.
print("\nPlease provide your Hugging Face API token:")
notebook_login()

# --- 3. GET TOKEN AND LOAD ALL DATASETS ---
print("\nReading cached token...")
hf_token = get_token()
if hf_token is None:
    raise Exception("Token not found! Please re-run the cell.")
else:
    print("Successfully read cached login token.")

print("\nLoading all 6 datasets...")
# Primary Datasets (using explicit auth and correct paths)
wildchat_dataset = load_dataset("allenai/WildChat", token=hf_token)
print("Loaded WildChat (from allenai/WildChat).")
lmsys_dataset = load_dataset("lmsys/lmsys-chat-1m", token=hf_token)
print("Loaded LMSYS-Chat-1M.")

# Probes (using correct, script-free paths)
rtp_dataset = load_dataset("allenai/real-toxicity-prompts")
print("Loaded RealToxicityPrompts.")
holistic_bias_dataset = load_dataset("fairnlp/holistic-bias", data_files=["sentences.csv"])
print("Loaded HolisticBias.")
bbq_dataset = load_dataset("walledai/BBQ")
print("Loaded BBQ.")
mgsm_dataset = load_dataset("sbintuitions/MGSM_en")
print("Loaded MGSM.")
print("--- ALL DATASET RELOADS COMPLETE. ---")


# --- 4. EXECUTE EMBEDDING TASK (SEC 4.3) ---
print("\nLoading sentence embedding model (all-mpnet-base-v2)...")
model = SentenceTransformer('all-mpnet-base-v2')
print("Model loaded.")

# Extract prompts
print(f"Extracting prompts from {len(wildchat_dataset['train'])} conversations...")
all_prompts = [
    conv[0]['content']
    for conv in wildchat_dataset['train']['conversation']
    if conv and len(conv) > 0 and conv[0]['role'] == 'user'
]
print(f"Total first-turn user prompts extracted: {len(all_prompts)}")

# Generate embeddings (This is the ~2 hour GPU task)
print("Generating embeddings for all prompts...")
prompt_embeddings = model.encode(all_prompts, show_progress_bar=True)
print("Embedding generation complete.")

# Save prompt embeddings to a file
np.save('prompt_embeddings.npy', prompt_embeddings)
print("Embeddings saved to 'prompt_embeddings.npy'.")


# --- 5. EXECUTE UMAP + CLUSTERING (SEC 4.3) ---
# This is the step that failed before, but will now succeed on the High-RAM machine.

print("\nStarting UMAP dimensionality reduction (768 -> 50)...")
start_time = time.time()
umap_reducer = umap.UMAP(
    n_neighbors=15,
    n_components=50,
    metric='cosine',
    random_state=42,
    verbose=True
)
umap_embeddings = umap_reducer.fit_transform(prompt_embeddings)
end_time = time.time()
print(f"UMAP reduction complete in {(end_time - start_time) / 60:.2f} minutes.")
np.save('umap_embeddings_50d.npy', umap_embeddings)
print("Saved 50-dimension UMAP embeddings.")

print("\nStarting HDBSCAN clustering...")
start_time = time.time()
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=100,  # Only find clusters with at least 100 prompts
    min_samples=15,
    metric='euclidean'
)
cluster_labels = clusterer.fit_predict(umap_embeddings)
end_time = time.time()
print(f"HDBSCAN complete in {(end_time - start_time) / 60:.2f} minutes.")

# --- 6. SAVE FINAL COHORT DATA ---
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise = np.sum(cluster_labels == -1)

print(f"\n--- CLUSTERING RESULTS (SEC 4.3 COMPLETE) ---")
print(f"Total cohorts discovered: {n_clusters}")
print(f"Total 'noise' points (outliers): {n_noise} ({(n_noise / len(cluster_labels) * 100):.2f}%)")

print("\nSaving final cohort data...")
df_prompts = pd.DataFrame({'prompt_text': all_prompts})
df_prompts['cluster_id'] = cluster_labels
df_prompts.to_csv('prompts_with_cohorts.csv', index=False)

print("\n--- ANALYSIS PIPELINE COMPLETE ---")

Installing dependencies...

Please provide your Hugging Face API token:


/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.



Reading cached token...
Successfully read cached login token.

Loading all 6 datasets...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00006.parquet:   0%|          | 0.00/291M [00:00<?, ?B/s]

...

Mon Sep  8 04:13:16 2025 Construct fuzzy simplicial set
Mon Sep  8 04:13:17 2025 Finding Nearest Neighbors
Mon Sep  8 04:13:17 2025 Building RP forest with 41 trees
Mon Sep  8 04:15:23 2025 NN descent for 19 iterations
	 1  /  19
	 2  /  19
	 3  /  19
	 4  /  19
	 5  /  19
	 6  /  19
	Stopping threshold met -- exiting after 6 iterations
Mon Sep  8 04:17:22 2025 Finished Nearest Neighbor Search
Mon Sep  8 04:17:29 2025 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Mon Sep  8 06:01:19 2025 Finished embedding
UMAP reduction complete in 108.11 minutes.
Saved 50-dimension UMAP embeddings.

Starting HDBSCAN clustering...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN complete in 26.00 minutes.

--- CLUSTERING RESULTS (SEC 4.3 COMPLETE) ---
Total cohorts discovered: 611
Total 'noise' points (outliers): 224698 (42.44%)

Saving final cohort data...

--- ANALYSIS PIPELINE COMPLETE ---


In [ ]:
import pandas as pd
import numpy as np

print("Loading final cohort data from 'prompts_with_cohorts.csv'...")
df_cohorts = pd.read_csv('prompts_with_cohorts.csv')

# --- 1. Filter out noise points ---
# Our diversity calculation is only for the discovered, valid cohorts.
valid_cohorts = df_cohorts[df_cohorts['cluster_id'] != -1]
print(f"Total valid (non-noise) prompts: {len(valid_cohorts)}")

# --- 2. Calculate Proportions (pk) ---
cohort_counts = valid_cohorts['cluster_id'].value_counts()
total_valid_points = len(valid_cohorts)
proportions = cohort_counts / total_valid_points
K = len(proportions)  # This is our K=611

# --- 3. Implement Hill-Chao Diversity (as per Sec 2.3.1 & 3.2) ---

# For q=1 (Shannon-based, order 1 diversity)
# This is the exponential of Shannon entropy.
shannon_entropy = -np.sum(proportions * np.log(proportions))
qD_1 = np.exp(shannon_entropy)

# For q=2 (Simpson-based, order 2 diversity)
# This is the inverse of the Simpson index.
simpson_index = np.sum(proportions**2)
qD_2 = 1 / simpson_index

print("\n--- Hill-Chao Calculation ---")
print(f"Total Cohorts (K): {K}")
print(f"Effective Diversity (q=1, Shannon): {qD_1:.4f}")
print(f"Effective Diversity (q=2, Simpson): {qD_2:.4f}")

# --- 4. Normalize Scores and Average (as per Sec 3.2) ---
# Normalized score s_D = (qD - 1) / (K - 1) [cite: 275]

s_D1_normalized = (qD_1 - 1) / (K - 1)
s_D2_normalized = (qD_2 - 1) / (K - 1)

# Final score is the average of the two [cite: 277]
s_D_final = (s_D1_normalized + s_D2_normalized) / 2

print("\n--- PILLAR 1 SCORE: DIVERSITY (s_D) ---")
print(f"Normalized Shannon Score (q=1): {s_D1_normalized:.6f}")
print(f"Normalized Simpson Score (q=2): {s_D2_normalized:.6f}")
print(f"--------------------------------------------------")
print(f"Final Diversity Pillar Score (s_D): {s_D_final:.6f}")

Loading final cohort data from 'prompts_with_cohorts.csv'...
Total valid (non-noise) prompts: 304730

--- Hill-Chao Calculation ---
Total Cohorts (K): 611
Effective Diversity (q=1, Shannon): 184.1926
Effective Diversity (q=2, Simpson): 38.5386

--- PILLAR 1 SCORE: DIVERSITY (s_D) ---
Normalized Shannon Score (q=1): 0.300316
Normalized Simpson Score (q=2): 0.061539
--------------------------------------------------
Final Diversity Pillar Score (s_D): 0.180927


In [ ]:
import pandas as pd
import numpy as np

print("Loading cohort data and extracting engagement metrics...")

# Load our primary cohort file
df_cohorts = pd.read_csv('prompts_with_cohorts.csv')

# --- 1. Get Conversation Lengths (Utility Metric) ---
# This list corresponds 1:1 with our prompts.
convo_lengths = [
    len(conv)
    for conv in wildchat_dataset['train']['conversation']
    if conv and len(conv) > 0 and conv[0]['role'] == 'user'
]

# Add the utility metric to our dataframe
df_cohorts['convo_length'] = convo_lengths

# --- 2. Calculate Per-Cohort Stats ---
# Filter out noise points (ID = -1)
valid_cohorts = df_cohorts[df_cohorts['cluster_id'] != -1].copy()

# Group by our 611 cohorts to get their size and mean utility
cohort_stats = valid_cohorts.groupby('cluster_id')['convo_length'].agg(
    cohort_size='count',
    mean_utility='mean'
).reset_index()

# --- 3. Calculate Prevalence and Weighted Stats ---
total_valid_points = cohort_stats['cohort_size'].sum()
cohort_stats['prevalence'] = cohort_stats['cohort_size'] / total_valid_points # This is our pi_k

# Overall prevalence-weighted mean utility
weighted_mean_utility = np.sum(cohort_stats['prevalence'] * cohort_stats['mean_utility']) # This is theta_bar_m

# Prevalence-weighted variance of utility
weighted_variance = np.sum(cohort_stats['prevalence'] * (cohort_stats['mean_utility'] - weighted_mean_utility)**2)
weighted_std_dev = np.sqrt(weighted_variance)

# --- 4. Calculate Equity Sub-Metrics (as per Sec 3.3.1) ---

# a) Dispersion (Weighted Coefficient of Variation)
# This measures the *spread* of utility relative to the mean. Lower is better.
dispersion_cv = weighted_std_dev / weighted_mean_utility
dispersion_score = 1.0 - dispersion_cv  # Invert so higher is better

# b) Worst-Group Score (WGS)
# This finds the lowest-utility cohort and measures it against the mean.
min_utility = cohort_stats['mean_utility'].min()
wgs_score = min_utility / weighted_mean_utility # Normalized against the weighted mean

# --- 5. Calculate Final Connectivity-Equity Score ---
# We use the formula from  with lambda=0.5 for an even balance
lambda_weight = 0.5
s_CE_final = (lambda_weight * wgs_score) + ((1 - lambda_weight) * dispersion_score)

print(f"\n--- PILLAR 2 SUB-SCORE: EQUITY (s_C,E) ---")
print(f"Overall Weighted Mean Utility (Convo Length): {weighted_mean_utility:.4f} turns")
print(f"Worst Cohort Mean Utility: {min_utility:.4f} turns")
print(f"--------------------------------------------------")
print(f"Worst-Group Score (WGS): {wgs_score:.6f}")
print(f"Dispersion (1.0 - CV): {dispersion_score:.6f}")
print(f"--------------------------------------------------")
print(f"Final Connectivity-Equity Score (s_C,E): {s_CE_final:.6f}")

Loading cohort data and extracting engagement metrics...

--- PILLAR 2 SUB-SCORE: EQUITY (s_C,E) ---
Overall Weighted Mean Utility (Convo Length): 4.6238 turns
Worst Cohort Mean Utility: 2.0000 turns
--------------------------------------------------
Worst-Group Score (WGS): 0.432543
Dispersion (1.0 - CV): 0.643067
--------------------------------------------------
Final Connectivity-Equity Score (s_C,E): 0.537805


In [ ]:
import numpy as np
import pandas as pd

print("Extracting all first-turn assistant responses...")

# We must ensure we extract the response that PAIRS with each prompt we embedded.
# The logic here mirrors the prompt extraction logic exactly.
all_responses = []
for conv in wildchat_dataset['train']['conversation']:
    # Check that the convo has at least 2 turns (user, assistant)
    # and that the first turn is 'user' (which we already filtered for)
    # and the second turn is 'assistant'.
    if (conv and
        len(conv) > 1 and
        conv[0]['role'] == 'user' and
        conv[1]['role'] == 'assistant'):

        all_responses.append(conv[1]['content'])
    else:
        # This should not happen if our prompt list is correct,
        # but as a safeguard, add an empty string to keep the lists aligned.
        # (Our previous extraction found 529,428 valid prompts, so this should match)
        all_responses.append("")

# Sanity check lengths
print(f"Total prompts extracted previously: 529428")
print(f"Total responses extracted now: {len(all_responses)}")
if len(all_responses) != 529428:
    print("WARNING: Mismatch in prompt/response pairs. Halting.")
else:
    print("Prompt/response pairs aligned. Proceeding to embed responses.")

    # --- This is the main computational step ---
    # We use the same 'model' variable that is still in memory.
    print("Generating embeddings for all responses... (This will take ~1.5-2 hours)")

    response_embeddings = model.encode(all_responses, show_progress_bar=True)

    print("Response embedding generation complete.")

    # Save the results
    np.save('response_embeddings.npy', response_embeddings)
    print("Response embeddings saved to 'response_embeddings.npy'.")

Extracting all first-turn assistant responses...
Total prompts extracted previously: 529428
Total responses extracted now: 529428
Prompt/response pairs aligned. Proceeding to embed responses.
Generating embeddings for all responses... (This will take ~1.5-2 hours)


Batches:   0%|          | 0/16545 [00:00<?, ?it/s]

Response embedding generation complete.
Response embeddings saved to 'response_embeddings.npy'.


In [ ]:
!pip install vendi-score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for vendi-score: filename=vendi_score-0.0.3-py3-none-any.whl size=13359 sha256=68a05c1ed8988709e2f7aecb09770836de6c02345b0d4b14b5c133d887b019c1
  Stored in directory: /root/.cache/pip/wheels/98/34/ea/4ee3b82c7bea904325df3721bacba64850db3a69c25a3097c3
Successfully built vendi-score


In [ ]:
!pip uninstall -y vendi
!pip install vendi-score

Found existing installation: vendi 0.1.24
Uninstalling vendi-0.1.24:
  Successfully uninstalled vendi-0.1.24
  Using cached vendi_score-0.0.3-py3-none-any.whl


In [ ]:
import numpy as np
from vendi_score import vendi   # <-- Correct package and module
import pandas as pd

print("Loading prompt and response embeddings...")
prompt_embeddings = np.load('prompt_embeddings.npy')
response_embeddings = np.load('response_embeddings.npy')

# --- 1. Sample the Data ---
SAMPLE_SIZE = 20000
total_size = prompt_embeddings.shape[0]
if SAMPLE_SIZE > total_size:  # Guard against sampling error
    SAMPLE_SIZE = total_size

rng = np.random.default_rng(42)  # Use modern Numpy generator
sample_indices = rng.choice(total_size, SAMPLE_SIZE, replace=False)

X_sample = prompt_embeddings[sample_indices]
Y_sample = response_embeddings[sample_indices]

print(f"Sampled {SAMPLE_SIZE} paired embeddings.")

# --- 2. Calculate Vendi Scores (fast path for embeddings) ---
# Use score_dual for the fast O(nd^2) covariance method
print("Calculating Vendi Score (dual) for Prompts (Set X)...")
vendi_X = vendi.score_dual(X_sample, normalize=True)
print("Calculating Vendi Score (dual) for Responses (Set Y)...")
vendi_Y = vendi.score_dual(Y_sample, normalize=True)

# --- 3. Amplification Score (s_C,K) ---
eps = 1e-12  # Numerical stability guard for log/division
delta_K = np.log((vendi_Y + eps) / (vendi_X + eps))

def sigmoid(x): return 1 / (1 + np.exp(-x))
s_CK_final = sigmoid(delta_K)

print("\n--- PILLAR 2 SUB-SCORE: CREATIVITY (s_C,K) ---")
print(f"Vendi Score (Prompts, X): {vendi_X:.4f}")
print(f"Vendi Score (Responses, Y): {vendi_Y:.4f}")
print(f"Amplification Log-Ratio (Delta-K): {delta_K:.6f}")
print("--------------------------------------------------")
print(f"Final Creative Amplification Score (s_C,K): {s_CK_final:.6f}")

# --- 4. FINAL PILLAR 2 SCORE (s_C) ---
s_CE_score = 0.537805  # From our previous step
s_C_final = np.sqrt(s_CE_score * s_CK_final)  # Geometric mean

print("\n--- FINAL PILLAR 2 SCORE (STATIC) ---")
print(f"s_C = sqrt(s_C,E * s_C,K)")
print(f"s_C = sqrt({s_CE_score:.4f} * {s_CK_final:.4f})")
print(f"Final Connectivity Pillar Score (s_C): {s_C_final:.6f}")
print("\n--- STATIC BASELINE CALCULATION COMPLETE ---")

Loading prompt and response embeddings...
Sampled 20000 paired embeddings.
Calculating Vendi Score (dual) for Prompts (Set X)...
Calculating Vendi Score (dual) for Responses (Set Y)...

--- PILLAR 2 SUB-SCORE: CREATIVITY (s_C,K) ---
Vendi Score (Prompts, X): 145.6389
Vendi Score (Responses, Y): 151.3972
Amplification Log-Ratio (Delta-K): 0.038776
--------------------------------------------------
Final Creative Amplification Score (s_C,K): 0.509693

--- FINAL PILLAR 2 SCORE (STATIC) ---
s_C = sqrt(s_C,E * s_C,K)
s_C = sqrt(0.5378 * 0.5097)
Final Connectivity Pillar Score (s_C): 0.523560

--- STATIC BASELINE CALCULATION COMPLETE ---


In [22]:
import pandas as pd
import numpy as np

print("Loading previous cohort data and original WildChat data...")
df_cohorts = pd.read_csv('prompts_with_cohorts.csv')

print("Extracting timestamps and conversation lengths for all valid conversations...")

# --- 1. Extract Timestamps and Conversation Lengths ---
timestamps = []
convo_lengths = []

for conv_data in wildchat_dataset['train']:
    conv = conv_data['conversation']
    if (conv and
        len(conv) > 0 and
        conv[0]['role'] == 'user'):

        timestamps.append(conv_data.get('timestamp', None))
        convo_lengths.append(len(conv))

print(f"Original prompts in CSV: {len(df_cohorts)}")
print(f"Extracted timestamps: {len(timestamps)}")

# --- 2. Create and Save Final Longitudinal Dataset ---
if len(df_cohorts) == len(timestamps):
    print("Data alignment successful. Adding columns...")

    # Add the data (this column now contains datetime objects)
    df_cohorts['datetime_col'] = timestamps
    df_cohorts['convo_length'] = convo_lengths

    # --- THIS IS THE FIX ---
    # The data is already a datetime, so no conversion is needed.
    # We just rename the column to 'datetime' for clarity.
    df_cohorts.rename(columns={'datetime_col': 'datetime'}, inplace=True)

    # Save this new master file
    df_cohorts.to_csv('final_longitudinal_data.csv', index=False)

    print("\nSuccessfully created 'final_longitudinal_data.csv'.")
    print("This file now contains (prompt_text, cluster_id, convo_length, datetime).")
    print("\nData time range:")
    print(f"Start: {df_cohorts['datetime'].min()}")
    print(f"End:   {df_cohorts['datetime'].max()}")

else:
    print(f"CRITICAL ERROR: Length mismatch. Prompts ({len(df_cohorts)}) vs Timestamps ({len(timestamps)})")

Loading previous cohort data and original WildChat data...
Extracting timestamps and conversation lengths for all valid conversations...
Original prompts in CSV: 529428
Extracted timestamps: 529428
Data alignment successful. Adding columns...

Successfully created 'final_longitudinal_data.csv'.
This file now contains (prompt_text, cluster_id, convo_length, datetime).

Data time range:
Start: 2023-04-10 00:01:08+00:00
End:   2023-11-09 16:22:27+00:00


In [23]:
import pandas as pd
import numpy as np

# --- 1. Load the Master Longitudinal File ---
print("Loading 'final_longitudinal_data.csv'...")
df = pd.read_csv('final_longitudinal_data.csv')
# Must parse the datetime column correctly
df['datetime'] = pd.to_datetime(df['datetime'])
print("Load complete. Calculating time series...")

# Define our calculation functions based on the logic we already built
def calculate_diversity(data):
    """Calculates s_D score for a given dataframe chunk."""
    valid_data = data[data['cluster_id'] != -1]
    if len(valid_data) == 0:
        return 0, 0, 0, 0 # Return zeros if a month has no data

    counts = valid_data['cluster_id'].value_counts()
    props = counts / len(valid_data)
    K = len(props)
    if K <= 1:
        return 0, 0, 0, 0 # Cannot calculate diversity of 1 cohort

    # q=1 (Shannon)
    shannon_entropy = -np.sum(props * np.log(props))
    qD_1 = np.exp(shannon_entropy)
    s_D1_norm = (qD_1 - 1) / (K - 1)

    # q=2 (Simpson)
    simpson_index = np.sum(props**2)
    qD_2 = 1 / simpson_index
    s_D2_norm = (qD_2 - 1) / (K - 1)

    s_D_final = (s_D1_norm + s_D2_norm) / 2
    return s_D_final, K, qD_1, qD_2

def calculate_equity(data):
    """Calculates s_C,E score for a given dataframe chunk."""
    valid_data = data[data['cluster_id'] != -1]
    if len(valid_data) == 0:
        return 0, 0, 0, 0

    # Get per-cohort stats (for this time slice)
    cohort_stats = valid_data.groupby('cluster_id')['convo_length'].agg(
        size='count',
        mean_utility='mean'
    ).reset_index()

    if len(cohort_stats) <= 1:
        return 0, 0, 0, 0 # Cannot calculate equity of 1 cohort

    total_points = cohort_stats['size'].sum()
    cohort_stats['prevalence'] = cohort_stats['size'] / total_points

    # Calculate weighted stats
    weighted_mean_util = np.sum(cohort_stats['prevalence'] * cohort_stats['mean_utility'])
    weighted_variance = np.sum(cohort_stats['prevalence'] * (cohort_stats['mean_utility'] - weighted_mean_util)**2)
    weighted_std_dev = np.sqrt(weighted_variance)

    if weighted_mean_util == 0:
        return 0, 0, 0, 0

    # a) Dispersion
    dispersion_cv = weighted_std_dev / weighted_mean_util
    disp_score = 1.0 - dispersion_cv

    # b) WGS
    min_utility = cohort_stats['mean_utility'].min()
    wgs_score = min_utility / weighted_mean_util

    s_CE_final = (0.5 * wgs_score) + (0.5 * disp_score)
    return s_CE_final, weighted_mean_util, min_utility, disp_score

# --- 2. Group Data by Month and Apply Calculations ---
# Set datetime as the index so we can group by time
df.set_index('datetime', inplace=True)
# 'ME' groups by Month-End.
monthly_groups = df.groupby(pd.Grouper(freq='ME'))

results_list = []

print("Running calculations for each month...")
for month, month_data in monthly_groups:
    if len(month_data) < 100: # Skip months with trivial data (like the start/end scraps)
        continue

    # Run our two functions
    s_D, K, _, _ = calculate_diversity(month_data)
    s_CE, avg_util, min_util, disp = calculate_equity(month_data)

    results_list.append({
        "Month": month.strftime('%Y-%m'),
        "s_D (Diversity)": s_D,
        "s_C,E (Equity)": s_CE,
        "Total_Prompts": len(month_data),
        "Valid_Cohorts_K": K,
        "Avg_Utility": avg_util,
        "Min_Utility": min_util
    })

# --- 3. Display Final Time Series Table ---
df_timeseries = pd.DataFrame(results_list)
df_timeseries.set_index("Month", inplace=True)

print("\n--- TIME SERIES RESULTS (INPUT FOR PILLAR 3: AGILITY) ---")
print(df_timeseries.to_markdown(floatfmt=".4f"))

Loading 'final_longitudinal_data.csv'...
Load complete. Calculating time series...
Running calculations for each month...

--- TIME SERIES RESULTS (INPUT FOR PILLAR 3: AGILITY) ---
| Month   |   s_D (Diversity) |   s_C,E (Equity) |   Total_Prompts |   Valid_Cohorts_K |   Avg_Utility |   Min_Utility |
|:--------|------------------:|-----------------:|----------------:|------------------:|--------------:|--------------:|
| 2023-04 |            0.2106 |           0.4968 |      51431.0000 |          487.0000 |        5.8459 |        2.0000 |
| 2023-05 |            0.1418 |           0.5406 |     106947.0000 |          525.0000 |        5.3081 |        2.0000 |
| 2023-06 |            0.1722 |           0.5210 |     103147.0000 |          537.0000 |        5.2465 |        2.0000 |
| 2023-07 |            0.1882 |           0.5182 |      77698.0000 |          553.0000 |        4.1665 |        2.0000 |
| 2023-08 |            0.1642 |           0.5494 |      59927.0000 |          533.0000 |     

In [25]:
import pandas as pd
import numpy as np
from vendi_score import vendi  # <-- THIS IS THE CORRECT, FINAL IMPORT
import warnings

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# --- 1. Load ALL Master Data ---
print("Loading all master files (CSV + Embeddings)...")
df_long = pd.read_csv('final_longitudinal_data.csv')
df_long['datetime'] = pd.to_datetime(df_long['datetime'])

# Load our two massive embedding sets
prompt_embeddings = np.load('prompt_embeddings.npy')    # Set X
response_embeddings = np.load('response_embeddings.npy')  # Set Y
print("All data loaded.")

# --- 2. Set up Sampling and Loop ---
SAMPLE_SIZE_PER_MONTH = 10000
results_list = []
rng = np.random.default_rng(42)
eps = 1e-12

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Group by month
monthly_groups = df_long.groupby(pd.Grouper(key='datetime', freq='ME'))

print(f"Calculating Creative Amplification (Vendi) for {len(monthly_groups)} months...")

for month, month_data in monthly_groups:
    if len(month_data) < 1000:
        continue

    current_indices = month_data.index.values

    n_sample = min(SAMPLE_SIZE_PER_MONTH, len(current_indices))
    sample_indices = rng.choice(current_indices, n_sample, replace=False)

    X_month_sample = prompt_embeddings[sample_indices]
    Y_month_sample = response_embeddings[sample_indices]

    # --- 4. Run Vendi Calculation (using the correct function call) ---
    vendi_X = vendi.score_dual(X_month_sample, normalize=True)
    vendi_Y = vendi.score_dual(Y_month_sample, normalize=True)

    # --- 5. Calculate Score ---
    delta_K = np.log((vendi_Y + eps) / (vendi_X + eps))
    s_CK_final = sigmoid(delta_K)

    results_list.append({
        "Month": month.strftime('%Y-%m'),
        "s_C,K (Creativity)": s_CK_final,
        "Delta_K_Ratio": delta_K,
        "Vendi_X": vendi_X,
        "Vendi_Y": vendi_Y
    })

# --- 6. Display Time Series Table ---
df_creativity_timeseries = pd.DataFrame(results_list)
df_creativity_timeseries.set_index("Month", inplace=True)

print("\n--- TIME SERIES RESULTS (CREATIVITY s_C,K) ---")
print(df_creativity_timeseries.to_markdown(floatfmt=".4f"))

Loading all master files (CSV + Embeddings)...
All data loaded.
Calculating Creative Amplification (Vendi) for 8 months...

--- TIME SERIES RESULTS (CREATIVITY s_C,K) ---
| Month   |   s_C,K (Creativity) |   Delta_K_Ratio |   Vendi_X |   Vendi_Y |
|:--------|---------------------:|----------------:|----------:|----------:|
| 2023-04 |               0.4937 |         -0.0251 |  159.3489 |  155.4011 |
| 2023-05 |               0.4929 |         -0.0283 |  156.0514 |  151.7021 |
| 2023-06 |               0.4963 |         -0.0147 |  151.4645 |  149.2479 |
| 2023-07 |               0.5207 |          0.0827 |  125.8351 |  136.6781 |
| 2023-08 |               0.5419 |          0.1680 |  111.6635 |  132.0930 |
| 2023-09 |               0.5392 |          0.1571 |  122.8387 |  143.7345 |
| 2023-10 |               0.5851 |          0.3438 |   87.0606 |  122.7870 |
| 2023-11 |               0.5415 |          0.1664 |  119.8094 |  141.4967 |


In [26]:
import pandas as pd
import numpy as np
from scipy import stats # For linear regression

# --- 1. Manually create our dataframes from the console output ---
# (This is faster than reloading and merging CSVs)

data_7_1 = {
    'Month': ['2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11'],
    's_D': [0.2106, 0.1418, 0.1722, 0.1882, 0.1642, 0.1294, 0.1010, 0.1177],
    's_CE': [0.4968, 0.5406, 0.5210, 0.5182, 0.5494, 0.5686, 0.5438, 0.5167]
}
df_main = pd.DataFrame(data_7_1).set_index('Month')

data_7_3 = {
    'Month': ['2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11'],
    's_CK': [0.4937, 0.4929, 0.4963, 0.5207, 0.5419, 0.5392, 0.5851, 0.5415]
}
df_creative = pd.DataFrame(data_7_3).set_index('Month')

# --- 2. Consolidate into Final Results Table ---
df_final = df_main.join(df_creative)

# Calculate final Pillar 2 score (Geometric Mean)
df_final['s_C'] = np.sqrt(df_final['s_CE'] * df_final['s_CK'])

print("--- FINAL MONTHLY TIME SERIES FOR ALL PILLARS ---")
print(df_final.to_markdown(floatfmt=".4f"))


# --- 3. Calculate Agility (Trend Component) ---
# We run a linear regression against time (using simple indices 0, 1, 2...)
time_steps = np.arange(len(df_final))

# a) Diversity Trend
reg_div = stats.linregress(time_steps, df_final['s_D'])
# b) Connectivity Trend
reg_con = stats.linregress(time_steps, df_final['s_C'])

print("\n\n--- AGILITY PILLAR (COMPONENT 1: TREND ANALYSIS) ---")
print("\nDiversity (s_D) Trend:")
print(f"  Slope: {reg_div.slope:.6f}")
print(f"  p-value: {reg_div.pvalue:.4f}")
if reg_div.pvalue < 0.05 and reg_div.slope < 0:
    print("  RESULT: Statistically significant downward trend (DECAY).")
elif reg_div.slope < 0:
    print("  RESULT: Downward trend (decay), but not statistically significant.")
else:
    print("  RESULT: Stable or positive trend.")

print("\nConnectivity (s_C) Trend:")
print(f"  Slope: {reg_con.slope:.6f}")
print(f"  p-value: {reg_con.pvalue:.4f}")
if reg_con.pvalue < 0.05 and reg_con.slope > 0:
    print("  RESULT: Statistically significant upward trend (IMPROVEMENT).")
elif reg_con.slope < 0:
     print("  RESULT: Downward trend (decay).")
else:
    print("  RESULT: Stable or positive trend (but not statistically significant).")

--- FINAL MONTHLY TIME SERIES FOR ALL PILLARS ---
| Month   |    s_D |   s_CE |   s_CK |    s_C |
|:--------|-------:|-------:|-------:|-------:|
| 2023-04 | 0.2106 | 0.4968 | 0.4937 | 0.4952 |
| 2023-05 | 0.1418 | 0.5406 | 0.4929 | 0.5162 |
| 2023-06 | 0.1722 | 0.5210 | 0.4963 | 0.5085 |
| 2023-07 | 0.1882 | 0.5182 | 0.5207 | 0.5194 |
| 2023-08 | 0.1642 | 0.5494 | 0.5419 | 0.5456 |
| 2023-09 | 0.1294 | 0.5686 | 0.5392 | 0.5537 |
| 2023-10 | 0.1010 | 0.5438 | 0.5851 | 0.5641 |
| 2023-11 | 0.1177 | 0.5167 | 0.5415 | 0.5290 |


--- AGILITY PILLAR (COMPONENT 1: TREND ANALYSIS) ---

Diversity (s_D) Trend:
  Slope: -0.011985
  p-value: 0.0198
  RESULT: Statistically significant downward trend (DECAY).

Connectivity (s_C) Trend:
  Slope: 0.007585
  p-value: 0.0213
  RESULT: Statistically significant upward trend (IMPROVEMENT).


In [27]:
!pip install statsmodels -q

In [28]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf # For Autocorrelation Function

# The df_final DataFrame is still in memory.

# --- 1. Calculate Variance ---
# High variance is a sign of instability.
var_D = df_final['s_D'].var()
var_C = df_final['s_C'].var()

# --- 2. Calculate Lag-1 Autocorrelation (AC(1)) ---
# AC(1) measures how much the score in one month predicts the next.
# A high positive value (near 1.0) is a classic EWI, signaling the system
# is "slowing down" and losing its ability to recover from perturbations.
ac1_D = acf(df_final['s_D'], nlags=1, fft=False)[1] # We only want the lag-1 value [index 1]
ac1_C = acf(df_final['s_C'], nlags=1, fft=False)[1]

print("--- AGILITY PILLAR (COMPONENT 2: EWI ANALYSIS) ---")
print("\nDiversity (s_D) Dynamics:")
print(f"  Variance:   {var_D:.6f}")
print(f"  Lag-1 AutoCorr: {ac1_D:.6f}")

print("\nConnectivity (s_C) Dynamics:")
print(f"  Variance:   {var_C:.6f}")
print(f"  Lag-1 AutoCorr: {ac1_C:.6f}")

--- AGILITY PILLAR (COMPONENT 2: EWI ANALYSIS) ---

Diversity (s_D) Dynamics:
  Variance:   0.001383
  Lag-1 AutoCorr: 0.310981

Connectivity (s_C) Dynamics:
  Variance:   0.000562
  Lag-1 AutoCorr: 0.510775
